In [ ]:
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from sklearn.cluster import KMeans

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout ,GlobalAveragePooling2D
from keras.applications import VGG16
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import cv2

data_path = r"Combined_images"
image_list = []
file_names = []

# Iterate over each file in the specified directory
for filename in os.listdir(data_path):
    if filename.endswith(('.jpg', '.png', '.tif', '.JPG')):
        file_path = os.path.join(data_path, filename)
        img = cv2.imread(file_path)
        # Append the image and its filename to the lists
        image_list.append(img)
        file_names.append(filename)

In [3]:
# Function to extract the number from the image filename
def extract_number_from_filename(filename):
    # Assuming the number is after 'combined_' and before the first '.'
    return filename.split('_')[1].split('.')[0]

In [4]:
clean_file_names = []
for name in file_names:
    clean_file_names.append(extract_number_from_filename(name))

In [5]:
import csv

# Sample data (replace these with your actual data)
images = image_list
image_names = clean_file_names

# Function to read the CSV and create a dictionary of image names to labels
def read_csv_to_dict(csv_file):
    df = pd.read_csv(csv_file)

    # Extract image names and labels
    image_names = df.iloc[:, 0].values
    labels = df.iloc[:, -1].values

    # Create a list of image-label pairs
    image_label_array = np.array([[name, label] for name, label in zip(image_names, labels)])
    
    # Convert the NumPy array to a dictionary
    image_label_dict = {row[0]: row[1] for row in image_label_array}
    return image_label_dict


# Function to create the list of images and their corresponding labels
def create_image_label_list(images, image_names, image_label_dict):
    image_label_list = []
    for image, name in zip(images, image_names):
        if name in image_label_dict:
            label = image_label_dict[name]
            image_label_list.append([name,image, label])
    return image_label_list

# Main execution
csv_file_path = 'image_labels.csv'  # Replace with your actual CSV file path
image_label_dict = read_csv_to_dict(csv_file_path)
image_label_list = create_image_label_list(images, image_names, image_label_dict)

# Output the result
for item in image_label_list:
    print(item)

['150007', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8), 'al-Mansuriyah']
['150008', array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 

## Trying to split while maintaining a balance of labels

In [32]:
import random

def split_data(image_label_list, test_size=0.2):
    """
    Splits image data into train and test sets while maintaining label distribution.

    Args:
        image_label_list: List of [image_number, image_data, label].
        test_size: Proportion of data to allocate to the test set (default: 0.2).

    Returns:
        train_data, test_data: Lists of [image_number, image_data, label].
    """

    # קיבוץ תמונות לפי תווית
    label_groups = {}
    for item in image_label_list:
        label = item[2]
        label_groups.setdefault(label, []).append(item)

    # חלוקה לפי תווית תוך שמירה על פרופורציות
    train_data, test_data = [], []
    for label, group in label_groups.items():
        random.shuffle(group)  # ערבוב כדי להבטיח אקראיות
        split_index = int(len(group) * test_size)
        test_data.extend(group[:split_index])
        train_data.extend(group[split_index:])

    return train_data, test_data

# דוגמה לשימוש
# train_data, test_data = split_data(image_label_list)

def split_data_1(df, test_size=0.2):
    """Splits a DataFrame into train and test sets while maintaining label distribution."""

    label_groups = df.groupby("label")

    train_data, test_data = pd.DataFrame(), pd.DataFrame()
    for label, group in label_groups:
        group = group.sample(frac=1)  # Shuffle the group
        split_index = int(len(group) * test_size)
        test_data = pd.concat([test_data, group[:split_index]])
        train_data = pd.concat([train_data, group[split_index:]])

    return train_data, test_data



In [63]:
def split_data_weighted(image_label_list, test_size=0.2):
    """
    Splits image data into train and test sets with weighted sampling.

    Args:
        image_label_list: List of lists, where each inner list is in the format [image_path, label].
        test_size: Proportion of data to allocate to the test set (default: 0.2).

    Returns:
        train_data, test_data: Lists of lists, each in the format [image_path, label].
    """
    # Group images by label
    label_groups = {}
    for item in image_label_list:
        label = item[1]  # Assuming label is at index 1 (after image_path)
        label_groups.setdefault(label, []).append(item)

    train_data, test_data = [], []
    for label, group in label_groups.items():
        weight = 1 / len(group)  # Inverse weight based on label frequency
        weighted_group = [(item, weight) for item in group]
        random.shuffle(weighted_group)  # Shuffle for randomness

        split_index = int(len(group) * test_size)
        test_data.extend([item for item, _ in weighted_group[:split_index]])
        train_data.extend([item for item, _ in weighted_group[split_index:]])

    return train_data, test_data

In [30]:
len(train_data)
len(test_data)
image_label_list

[['150007',
  array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         ...,
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8),
  'al-Mansuriyah'],
 ['150008',
  array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0

 Check if all labels are present in the training set

In [11]:
# Extract unique labels from train_data
train_labels = set(item[2] for item in train_data)

# Extract unique labels from the original dataset
all_labels = set(item[2] for item in image_label_list)

# Check if all labels are present in the training set
if all_labels.issubset(train_labels):
    print("כל התוויות נמצאות בקבוצת האימון!")
else:
    missing_labels = all_labels - train_labels
    print("חסרות התוויות הבאות בקבוצת האימון:", missing_labels)

כל התוויות נמצאות בקבוצת האימון!


before building the model...

In [53]:
import tempfile

# image_label_list is in the format: [[image_number, image_data, label], ...]

# Create temporary directory to store images
temp_dir = tempfile.mkdtemp()

# Convert image arrays to image files and create DataFrame
image_paths = []
for image_number, image_data, label in image_label_list:
    image = Image.fromarray(image_data)
    image_path = os.path.join(temp_dir, f"{image_number}.jpg")
    image.save(image_path)
    image_paths.append(image_path)


In [65]:

df = pd.DataFrame({"image_path": image_paths, "label": [item[2] for item in image_label_list]})

# Normalize labels: Treat variations as the same label {'al-Mansuriyah'= 'al-Mansuriyah?'='(al-Mansuriyah)'}
df['label'] = df['label'].astype(str).str.replace(r'[?()]', '', regex=True).str.strip()

# # Get top 8 most frequent labels from the filtered DataFrame
# top_labels = df_filtered['label'].value_counts().head(8).index.tolist()

# # Filter DataFrame to keep only top labels
# filtered_df = df_filtered[df_filtered['label'].isin(top_labels)]

# # Split DataFrame into train and validation sets
# train_df, val_df = split_data_1(filtered_df)  # Using the defined function

# # Convert back to DataFrames
# train_df = pd.DataFrame(train_df, columns=["image_path", "label"])
# val_df = pd.DataFrame(val_df, columns=["image_path", "label"])

# Filter out 'uncertain' labels
df = df[df['label'] != 'uncertain']

# Splitting the Data
train_df, val_df = split_data_weighted(df.values.tolist())

# Convert back to DataFrames
train_df = pd.DataFrame(train_df, columns=["image_path", "label"])
val_df = pd.DataFrame(val_df, columns=["image_path", "label"])

# Finding unique labels
train_labels = set(train_df['label'])
val_labels = set(val_df['label'])

# Getting common labels
common_labels = train_labels.intersection(val_labels)

# Ensuring common labels in both DataFrames
train_df = train_df[train_df['label'].isin(common_labels)]
val_df = val_df[val_df['label'].isin(common_labels)]

# Getting the top 8 labels
value_counts = train_df['label'].value_counts()
top_labels = value_counts.head(8).index.tolist()

# Filtering both DataFrames to top 8 labels
train_df = train_df[train_df['label'].isin(top_labels)]
val_df = val_df[val_df['label'].isin(top_labels)]

# Display labels present in both dataframes
print(f"Labels present in both train and val sets: {common_labels}")

# Ensure the number of classes matches
num_classes = len(common_labels)  # Update num_classes for model



Labels present in both train and val sets: {'al-Mansuriyah', 'al-Mahdiyah', 'Misr', 'Atarablus', 'm.', 'Filastin'}


In [66]:
print(train_df['label'])

0      al-Mansuriyah
1      al-Mansuriyah
2      al-Mansuriyah
3      al-Mansuriyah
4      al-Mansuriyah
           ...      
324         Filastin
325         Filastin
326         Filastin
327         Filastin
328         Filastin
Name: label, Length: 328, dtype: object


In [67]:

# Create an ImageDataGenerator (add augmentation parameters if desired)
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values
    rotation_range=20,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'  # How to fill in newly created pixels
)

# Create generators from DataFrames (corrected x_col)
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image_path",  # Corrected column name
    y_col="label",  
    class_mode="categorical",
    target_size=(224, 224),  
    batch_size=32,  
)


val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="image_path",  # Corrected column name
    y_col="label",
    class_mode="categorical",
    target_size=(224, 224),
    batch_size=32
)


Found 328 validated image filenames belonging to 6 classes.
Found 80 validated image filenames belonging to 6 classes.


In [22]:

# # Create a DataFrame from your image_label_list
# df = pd.DataFrame(image_label_list, columns=["image_number", "image_data", "label"])

# # Split DataFrame into train and validation sets
# train_df, val_df = split_data(df.values.tolist())

# # Convert back to DataFrames
# train_df = pd.DataFrame(train_df, columns=["image_number", "image_data", "label"])
# val_df = pd.DataFrame(val_df, columns=["image_number", "image_data", "label"])


In [68]:


# num_classes = len(filtered_df['label'].unique())  

# Rebuild the model with the correct num_classes
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x) 
predictions = Dense(num_classes, activation='softmax')(x) 
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# אימון המודל
model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10


C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 247s 14s/step - accuracy: 0.4086 - loss: 3.0300 - val_accuracy: 0.4125 - val_loss: 691.4587
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 183s 14s/step - accuracy: 0.5509 - loss: 1.3127 - val_accuracy: 0.0500 - val_loss: 194.6411
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 182s 13s/step - accuracy: 0.5560 - loss: 1.1046 - val_accuracy: 0.3750 - val_loss: 1875.2809
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 189s 14s/step - accuracy: 0.6917 - loss: 0.9654 - val_accuracy: 0.3750 - val_loss: 165.0512
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 184s 13s/step - accuracy: 0.6957 - loss: 0.9075 - val_accuracy: 0.3750 - val_loss: 37.9965
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 181s 13s/step - accuracy: 0.7245 - loss: 0.7955 - val_accuracy: 0.3750 - val_loss: 2203.2148
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 180s 13s/step - accuracy: 0.7158 - loss: 0.9366 - val_accuracy: 0.3750 - val_loss: 32.0868
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 182s 14s/step - accuracy: 0.7146 - loss: 0.7842 - val_accuracy:

In [70]:

# Delete temporary directory after training (optional but recommended)
import shutil
shutil.rmtree(temp_dir)

In [72]:
# Create temporary directory to store images
temp_dir = tempfile.mkdtemp()

# Convert image arrays to image files and create DataFrame, excluding 'uncertain' labels
image_paths = []
labels = []
for image_number, image_data, label in test_data:
    if label != 'uncertain':
        # Normalize label (remove ?, (), and extra spaces)
        label = label.replace("?", "").replace("(", "").replace(")", "").strip()

        image = Image.fromarray(image_data)
        image_path = os.path.join(temp_dir, f"{image_number}.jpg")
        image.save(image_path)
        image_paths.append(image_path)
        labels.append(label)

test_df = pd.DataFrame({"image_path": image_paths, "label": labels})

In [73]:
test_df

,image_path,label
0,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
1,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
2,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
3,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
4,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
...,...,...
74,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mahdiyah
75,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
76,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,al-Mansuriyah
77,C:\Users\avoav\AppData\Local\Temp\tmpg0esihpa\...,Filastin


In [74]:
# Get unique labels from train_df and test_df
train_labels = set(train_df['label'])
test_labels = set(test_df['label'])

# Check if test labels are a subset of train labels
if test_labels.issubset(train_labels):
    print("All labels in the test set are also present in the training set.")
else:
    missing_labels = test_labels - train_labels
    print("The following labels are in the test set but not in the training set:", missing_labels)

All labels in the test set are also present in the training set.


In [75]:
# Create a test generator (similar to train and val generators)
test_datagen = ImageDataGenerator(rescale=1./255)  # Only rescale, no augmentation

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="image_path",  
    y_col="label",  
    class_mode="categorical",
    batch_size=32,           # Typically same batch size as train/val
    shuffle=False            # Crucial: Do NOT shuffle test data
)

Found 79 validated image filenames belonging to 5 classes.


In [76]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\asyncio\base_events.py", line 639, in run_forever

  File "C:\Users\avoav\AppData\Local\Programs\Python\Python312\Lib\asyncio

### same , just with Weighted Sampling.

In [1]:


# Load the CSV file
csv_file = 'image_labels.csv'
df = pd.read_csv(csv_file)

# Extract image names and labels
image_names = df.iloc[:, 0].values
labels = df.iloc[:, -1].values

# Create a list of image-label pairs
image_label_pairs = np.array([[name, label] for name, label in zip(image_names, labels)])

# Print the list of image-label pairs
for pair in image_label_pairs:
    print(pair)

# Assuming you want to plot images and labels
def plot_images_with_labels(image_label_pairs, image_dir, num_samples=5):
    unique_labels = np.unique(image_label_pairs[:, 1].astype(int))
    plt.figure(figsize=(20, 10))
    
    for i, label in enumerate(unique_labels):
        label_indices = np.where(image_label_pairs[:, 1].astype(int) == label)[0]
        sample_indices = np.random.choice(label_indices, num_samples, replace=False)
        
        for j, idx in enumerate(sample_indices):
            plt.subplot(len(unique_labels), num_samples, i * num_samples + j + 1)
            img_name = image_label_pairs[idx, 0]
            img_path = os.path.join(image_dir, img_name)
            img = load_img(img_path, target_size=(224, 224))
            plt.imshow(img)
            plt.title(f'Cluster {label}\n{os.path.basename(img_path)}')
            plt.axis('off')

# Set the directory where the images are located
image_dir = "Combined_images"

plot_images_with_labels(image_label_pairs, image_dir, num_samples=5)
plt.tight_layout()
plt.show()


['150007' 'al-Mansuriyah']
['150008' 'm.']
['150009' 'm.']
['150014' 'Misr']
['150015' 'Misr']
['150016' 'al-Mahdiyah']
['150017' 'Misr']
['150030' 'm.']
['150031' 'm.']
['150032' 'al-Mansuriyah']
['150033' 'Misr']
['150034' 'al-Mansuriyah']
['150036' 'm.']
['150037' 'Misr']
['150038' 'Misr']
['150039' 'al-Mahdiyah']
['150040' 'Misr']
['150041' 'm.']
['150042' 'Misr']
['150043' 'm.']
['150044' 'm.']
['150045' 'Misr']
['150046' '(Atarablus)']
['150047' '(Atarablus)']
['150048' 'al-Mansuriyah']
['150049' 'Misr']
['150050' 'Misr']
['150051' '(al-Mansuriyah)']
['150052' 'al-Mansuriyah']
['150053' 'al-Mansuriyah']
['150054' 'al-Mansuriyah']
['150055' 'Misr']
['150056' 'Misr']
['150057' 'al-Mansuriyah']
['150058' 'al-Mansuriyah']
['150059' 'Misr']
['150060' 'al-Mansuriyah']
['150061' 'Misr']
['150062' 'm.']
['150063' 'm.']
['150064' 'Misr']
['150066' 'al-Mansuriyah']
['150067' 'm.']
['150068' 'al-Mansuriyah']
['150069' 'al-Mansuriyah']
['150070' 'Dimashq']
['150071' 'al-Mansuriyah']
['150072

ValueError: invalid literal for int() with base 10: 'al-Mansuriyah'